## Задача 1

Реализовать класс для работы с линейной регрессией

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import time
import matplotlib.pyplot as plt

In [ ]:
class MyLinearRegression:
    """
    Parameters
    ----------
    regularization : {None, 'l1', 'l2', 'l1l2'}, default=None
        Какую регуляризацию добавить к модели. Если значение `None`, то без регуляризации.

    weight_calc : {'matrix', 'gd', 'sgd'}, default='matrix'
        Каким образом вычислять вектор весов: матрично ('matrix'), градиентным спуском ('gd') или стохастическим градиентным спуском ('sgd'). При этом, при 'l1' или 'l1l2' нельзя использовать параметр 'matrix'.

    Attributes
    ----------
    coefs_ : Вектор коэффициентов размера (p, 1), где p — количество признаков.
    intercept_ : Значение коэффициента, отвечающего за смещение
    """

    def __init__(self,
                 regularization=None,
                 weight_calc='matrix',
                 lambda_1=None,
                 lambda_2=None,
                 batch_size=20,
                 max_iter = 10000,
                 alpha=1e-2,
                 early_stopping = 0.001):
        if regularization not in [None, 'l1', 'l2', 'l1l2']:
            raise TypeError(f"Параметр regularization не может принимать значение '{regularization}'")
        if weight_calc not in ['matrix', 'gd', 'sgd']:
            raise TypeError(f"Параметр weight_calc не может принимать значение '{weight_calc}'")
        if regularization in ['l1', 'l1l2'] and lambda_1 is None:
            raise TypeError(f"Значение коэффициента регулризации l1 не задано")
        if regularization in ['l2', 'l1l2'] and lambda_2 is None:
            raise TypeError(f"Значение коэффициента регулризации l2 не задано")

        self.regularization = regularization
        self.weight_calc = weight_calc
        self.lambda_1 = lambda_1
        self.lambda_2 = lambda_2
        self.batch_size = batch_size
        self.coef_ = None
        self.intercept_ = None
        self.alpha = alpha
        self.max_iter = max_iter
        self.early_stopping = early_stopping

    def fit(self, X: pd.DataFrame, y: pd.DataFrame):

        if type(X) != np.ndarray:
            X = X.to_numpy()
        if type(y) != np.ndarray:
            y = y.to_numpy()

        if self.weight_calc == 'matrix':
            self.matrix_method(X, y, self.lambda_1, self.lambda_2)
        elif self.weight_calc == 'gd':
            self.gd(X, y)
        else:
            self.sgd(X, y)


    def matrix_method(self, X, y, lambda_1, lambda_2):
        if self.regularization in ['l1', 'l1l2']:
            raise TypeError("При 'l1' или 'l1l2' нельзя использовать параметр 'matrix'.")
        n,m = X.shape
        X = np.hstack([np.ones((n, 1)), X])
        y = y.reshape(-1, 1)

        if self.regularization == 'l2':
            reg = np.eye((X.T @ X).shape[0])
            theta = np.linalg.inv(X.T @ X + self.lambda_2 * reg) @ X.T @ y
        else:
            theta = np.linalg.inv(X.T @ X) @ X.T @ y

        self.intercept_ = theta[0, 0]
        self.coef_ = theta[1:, 0]

    def gd(self, X, y):

        n, m = X.shape

        # Добавляем столбец единиц для смещения (intercept)
        X = np.hstack([np.ones((n, 1)), X])  # добавляем столбец единичных значений для intercept
        weights = np.random.randn(X.shape[1]) * 0.01  # Инициализация весов случайными малыми значениями

        for i in tqdm(range(self.max_iter)):
            # Предсказания
            predictions = X.dot(weights)

            # Ошибка
            error = predictions - y.flatten()

            # Градиент
            gradients = (2 / n) * X.T.dot(error)

            # Регуляризация
            if self.regularization == 'l1':
                gradients += self.lambda_1 * np.sign(weights)
            if self.regularization == 'l2':
                gradients += 2 * self.lambda_2 * weights
            if self.regularization == 'l1l2':
                gradients += self.lambda_1 * np.sign(weights)
                gradients += 2 * self.lambda_2 * weights

            # Обновляем веса
            weights -= self.alpha * gradients

            # Проверка на малый градиент (early stopping)
            if np.linalg.norm(gradients) < self.early_stopping:
                print(f"Early stopping at iteration {i}")
                break

            # Проверка на NaN в весах и ошибках
            if np.any(np.isnan(weights)) or np.any(np.isnan(error)):
                print(f"NaN encountered at iteration {i}. Stopping.")
                break

            # Проверка на слишком большие значения весов
            if np.any(np.abs(weights) > 1e10):
                print(f"Warning: weights are too large at iteration {i}")
                break

        self.intercept_ = weights[0]  # Первый элемент - это intercept
        self.coef_ = weights[1:]  # Остальные элементы - это коэффициенты для признаков


    def sgd(self, X, y):
        n,m = X.shape

        X = np.hstack([np.ones((n, 1)), X])  # добавляем столбец единичных значений для intercept
        weights = np.random.randn(X.shape[1]) * 0.01

        for i in tqdm(range(self.max_iter)):
            batch_indices = np.random.choice(n, self.batch_size, replace=False)
            X_batch = X[batch_indices]
            y_batch = y[batch_indices]

            #Предсказания
            predictions = X_batch.dot(weights)

            #Ошибка
            error = predictions - y_batch#.flatten()

            #градиент
            gradients = 2 * X_batch.T.dot(error) / self.batch_size

            if self.regularization == 'l1':
                gradients += self.lambda_1 * np.sign(weights)
            if self.regularization == 'l2':
                gradients += 2 * self.lambda_2 * weights
            if self.regularization == 'l1l2':
                gradients += self.lambda_1 * np.sign(weights)
                gradients += 2 * self.lambda_2 * weights

            #обновляем веса
            weights -= self.alpha * gradients

            if np.any(np.isnan(weights)) or np.any(np.isnan(error)):
                print(f"NaN encountered at iteration {i}. Stopping.")
                break

            if np.any(np.abs(weights) > 1e10):
                print(f"Warning: weights are too large at iteration {i}")
                break

            #Проверка, что градиент не маленький
            if np.linalg.norm(gradients) < self.early_stopping:
                print(f"Early stopping at iteration {i}")
                break

        self.intercept_ = weights[0]
        self.coef_ = weights[1:]

    def predict(self, X: np.array, ss=True):
        # print("coef_.shape = ", self.coef_.shape)
        # print("intercept_.shape = ", self.intercept_.shape)
        # print("X= ", X)
        # print("coeff= ", self.coef_)
        # print("intercept= ", self.intercept_)

        # print("type X= ", type(X))
        # print("type coeff= ", type(self.coef_))
        # print("type intercept= ", type(self.intercept_))

        # print("shape X= ", X.shape)
        # print("shape coeff= ", self.coef_.shape)
        # print("shape intercept= ", self.intercept_.shape)

        X_arr = np.array(X, dtype=float)

        if ss:
            return X_arr @ np.asarray(self.coef_) + np.asarray(self.intercept_)
        return X_arr @ np.asarray(self.coef_)

    def score_MSE(self, X: np.array, y: np.array):
        y_pred = self.predict(X)
        return np.mean((y_pred - y) ** 2)

    def score_MAE(self, X: np.array, y: np.array):
        y_pred = self.predict(X)
        return np.mean(np.abs(y_pred - y))

Используя датасет про автомобили (целевой признак — price), сравнить (качество, скорость обучения и предсказания, важность признаков) модели `MyLinearRegression` с различными гиперпараметрами, сделать выводы.

# Задача 2

[Соревнование на Kaggle](https://www.kaggle.com/competitions/stat-plus-ml-2024)

In [ ]:
!unzip /content/ensembles-competition.zip

Archive:  /content/ensembles-competition.zip
  inflating: Sample_Submission.csv   
  inflating: for_prediction.csv      
  inflating: train_contest.csv       


In [ ]:
!pip install unrar
!unrar x /content/kaggle_linreg.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/kaggle_linreg.rar

Extracting  for_prediction.csv                                             0%  1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20%  OK 
Extracting  Sample_Submission.csv                                         20%  OK 
Extracting  train_contest.csv                                             20% 21% 22% 23% 24% 25% 26% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38% 39% 40% 41% 42% 43% 44% 45% 46% 47% 48% 49% 50% 51% 52% 53% 54% 55% 56% 57% 58% 59% 60% 61% 62% 63% 64% 65% 66% 67% 68% 69% 70% 71% 72% 73% 74% 75% 76% 77

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import time
import matplotlib.pyplot as plt

#Предобработка данных

In [ ]:
data_train_orig = pd.read_csv('train_contest.csv')
data_for_pred_orig = pd.read_csv('for_prediction.csv')

data_train = data_train_orig.copy()
data_for_pred = data_for_pred_orig.copy()

In [ ]:
data_train_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30072 entries, 0 to 30071
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   premium                   30072 non-null  bool   
 1   name                      30072 non-null  object 
 2   department                752 non-null    object 
 3   has_test                  30072 non-null  bool   
 4   response_letter_required  30072 non-null  bool   
 5   area                      30072 non-null  object 
 6   type                      30072 non-null  object 
 7   address                   19105 non-null  object 
 8   response_url              0 non-null      float64
 9   sort_point_distance       0 non-null      float64
 10  published_at              30072 non-null  object 
 11  created_at                30072 non-null  object 
 12  archived                  30072 non-null  bool   
 13  insider_interview         94 non-null     object 
 14  url   

In [ ]:
data_train_orig

,premium,name,department,has_test,response_letter_required,area,type,address,response_url,sort_point_distance,...,working_time_intervals,working_time_modes,accept_temporary,description,experience,key_skills,specializations,region,immediate_redirect_url,mean_salary
0,False,Агент по недвижимости,NaN,False,False,"{'id': '2', 'name': 'Санкт-Петербург', 'url': ...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Санкт-Петербург', 'street': 'Невский...",NaN,NaN,...,[],[],False,"<p>Мы ищем людей, которым не достаточно того, ...","{'id': 'noExperience', 'name': 'Нет опыта'}","[{'name': 'Жилая недвижимость'}, {'name': 'Акт...","[{'id': '20.20', 'name': 'Агент', 'profarea_id...",Санкт-Петербург,NaN,125000.0
1,False,IT-специалист/ техник,NaN,False,False,"{'id': '1130', 'name': 'Братск', 'url': 'https...","{'id': 'open', 'name': 'Открытая'}",NaN,NaN,NaN,...,[],[],False,<ul> </ul> <p><em><strong>Навыки</strong></em>...,"{'id': 'between1And3', 'name': 'От 1 года до 3...",[],"[{'id': '1.172', 'name': 'Начальный уровень, М...",Иркутская область,NaN,50000.0
2,False,Старший/ведущий инженер-программист,NaN,False,False,"{'id': '76', 'name': 'Ростов-на-Дону', 'url': ...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Ростов-на-Дону', 'street': '60К-9, 1...",NaN,NaN,...,[],[],False,<strong>Обязанности:</strong> <ul> <li>Разрабо...,"{'id': 'between1And3', 'name': 'От 1 года до 3...","[{'name': 'Linux'}, {'name': 'Bash'}, {'name':...","[{'id': '1.221', 'name': 'Программирование, Ра...",Ростовская область,NaN,80000.0
3,False,Эксперт / методист приемной комиссии,NaN,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Москва', 'street': '1-я Миусская ули...",NaN,NaN,...,[],[],False,<p>Департамент информационных технологий РХТУ ...,"{'id': 'between1And3', 'name': 'От 1 года до 3...","[{'name': '1С программирование'}, {'name': 'Об...","[{'id': '3.150', 'name': 'Менеджмент продукта ...",Москва,NaN,120000.0
4,False,Табельщица/табельщик,NaN,False,False,"{'id': '1613', 'name': 'Энгельс', 'url': 'http...","{'id': 'open', 'name': 'Открытая'}","{'city': None, 'street': None, 'building': Non...",NaN,NaN,...,[],[],False,<p><strong>Обязанности:</strong></p> <ul> <li>...,"{'id': 'between1And3', 'name': 'От 1 года до 3...","[{'name': 'Составление отчетности'}, {'name': ...","[{'id': '18.142', 'name': 'Машиностроение', 'p...",Саратовская область,NaN,15000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30067,False,Руководитель отдела персонала,NaN,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Москва', 'street': 'Ленинградский пр...",NaN,NaN,...,[],[],False,<strong>Обязанности:</strong> <ul> <li>Управле...,"{'id': 'moreThan6', 'name': 'Более 6 лет'}","[{'name': 'Подбор персонала'}, {'name': 'Адапт...","[{'id': '9.738', 'name': 'Другое', 'profarea_i...",Москва,NaN,190000.0
30068,False,Специалист по развитию и обучению персонала,NaN,False,False,"{'id': '1641', 'name': 'Набережные Челны', 'ur...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Набережные Челны', 'street': 'проспе...",NaN,NaN,...,[],[],False,<p>Группа компаний &quot;Бринэкс&quot; приглаш...,"{'id': 'between1And3', 'name': 'От 1 года до 3...","[{'name': 'Управление временем'}, {'name': 'Ор...","[{'id': '6.254', 'name': 'Рекрутмент', 'profar...",Республика Татарстан,NaN,45000.0
30069,False,Инженер-программист,NaN,False,False,"{'id': '1399', 'name': 'Магнитогорск', 'url': ...","{'id': 'open', 'name': 'Открытая'}",NaN,NaN,NaN,...,[],[],False,<strong>Обязанности:</strong> <ul> <li>Разрабо...,"{'id': 'between1And3', 'name': 'От 1 года до 3...","[{'name': 'SQL'}, {'name': 'Работа с базами да...","[{'id': '1.221', 'name': 'Программирование, Ра...",Челябинская область,NaN,75000.0
30070,False,Куратор проекта,"{'id': '1947314-1947314-ancor', 'name': 'ANCOR'}",False,False,"{'id': '3', 'name': 'Екатеринбург', 'url': 'ht...","{'id': 'open', 'name': 'Откр

In [ ]:
data_train[(data_train['type'] !=  "{'id': 'open', 'name': 'Открытая'}")]['type']

,type
222,"{'id': 'anonymous', 'name': 'Анонимная'}"
259,"{'id': 'anonymous', 'name': 'Анонимная'}"
1538,"{'id': 'anonymous', 'name': 'Анонимная'}"
1754,"{'id': 'anonymous', 'name': 'Анонимная'}"
1870,"{'id': 'anonymous', 'name': 'Анонимная'}"
2020,"{'id': 'anonymous', 'name': 'Анонимная'}"
2058,"{'id': 'anonymous', 'name': 'Анонимная'}"
3158,"{'id': 'anonymous', 'name': 'Анонимная'}"
3196,"{'id': 'anonymous', 'name': 'Анонимная'}"
4300,"{'id': 'anonymous', 'name': 'Анонимная'}"


In [ ]:
obj_cols = data_train_orig.select_dtypes(object).columns

for col in obj_cols:
    print(col)
    print(f"Процент пропущенных значений в столбце {col} = {(data_train_orig[col] == '[]').sum()/data_train_orig.shape[0]}")
    print(f"Среднее значение зарплаты для пропущенных = {data_train_orig[data_train[col] == '[]']['mean_salary'].mean()}")
    print(f"Среднее значение зарплаты для присутствующих = {data_train_orig[data_train_orig[col] != '[]']['mean_salary'].mean()}")
    print()

name
Процент пропущенных значений в столбце name = 0.0
Среднее значение зарплаты для пропущенных = nan
Среднее значение зарплаты для присутствующих = 97728.25091447194

department
Процент пропущенных значений в столбце department = 0.0
Среднее значение зарплаты для пропущенных = nan
Среднее значение зарплаты для присутствующих = 97728.25091447194

area
Процент пропущенных значений в столбце area = 0.0
Среднее значение зарплаты для пропущенных = nan
Среднее значение зарплаты для присутствующих = 97728.25091447194

type
Процент пропущенных значений в столбце type = 0.0
Среднее значение зарплаты для пропущенных = nan
Среднее значение зарплаты для присутствующих = 97728.25091447194

address
Процент пропущенных значений в столбце address = 0.0
Среднее значение зарплаты для пропущенных = nan
Среднее значение зарплаты для присутствующих = 97728.25091447194

published_at
Процент пропущенных значений в столбце published_at = 0.0
Среднее значение зарплаты для пропущенных = nan
Среднее значение з

In [ ]:
bool_cols = data_train_orig.select_dtypes(bool).columns

for col in bool_cols:
    print(col)
    print(f"Процент false в столбце {col} = {(data_train_orig[col] == True).sum()/data_train_orig.shape[0]}")
    print(f"Среднее значение зарплаты для True = {data_train_orig[data_train_orig[col] == True]['mean_salary'].mean()}")
    print(f"Среднее значение зарплаты для False = {data_train_orig[data_train_orig[col] == False]['mean_salary'].mean()}")
    print()

premium
Процент false в столбце premium = 0.003391859537110934
Среднее значение зарплаты для True = 143156.86274509804
Среднее значение зарплаты для False = 97573.6390223557

has_test
Процент false в столбце has_test = 0.014032987496674648
Среднее значение зарплаты для True = 133779.65758293838
Среднее значение зарплаты для False = 97215.14151770658

response_letter_required
Процент false в столбце response_letter_required = 0.01646049481245012
Среднее значение зарплаты для True = 102348.42121212122
Среднее значение зарплаты для False = 97650.92784934239

archived
Процент false в столбце archived = 0.00013301409949454643
Среднее значение зарплаты для True = 68000.0
Среднее значение зарплаты для False = 97732.20571704138

accept_temporary
Процент false в столбце accept_temporary = 0.09364192604416068
Среднее значение зарплаты для True = 112245.21608664772
Среднее значение зарплаты для False = 96228.40596565894



Удалим сильно несбалансированные данные, ненужные данные и данные с большими пропусками

In [ ]:
bool_cols = data_train_orig.select_dtypes(bool).columns
unbalanced_data = bool_cols
data_with_lots_nans = ['working_days', 'working_time_intervals', 'working_time_modes', 'relations', 'sort_point_distance', 'insider_interview']
useless_data = ['department', 'response_url', 'immediate_redirect_url', 'type', 'published_at', 'created_at', 'url', 'alternate_url', "address", "contacts"]

In [ ]:
data_train = data_train_orig.drop([*unbalanced_data, *data_with_lots_nans, *useless_data], axis=1)

In [ ]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30072 entries, 0 to 30071
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             30072 non-null  object 
 1   area             30072 non-null  object 
 2   employer         30072 non-null  object 
 3   snippet          30072 non-null  object 
 4   schedule         30072 non-null  object 
 5   description      30072 non-null  object 
 6   experience       30072 non-null  object 
 7   key_skills       30072 non-null  object 
 8   specializations  30072 non-null  object 
 9   region           30072 non-null  object 
 10  mean_salary      30072 non-null  float64
dtypes: float64(1), object(10)
memory usage: 2.5+ MB


In [ ]:
data_train_orig[data_train_orig['contacts'].isnull()]['mean_salary'].describe()

,mean_salary
count,1.098700e+04
mean,1.013885e+05
std,9.300644e+04
min,6.250000e+01
25%,4.050000e+04
50%,7.000000e+04
75%,1.300000e+05
max,1.100000e+06


In [ ]:
data_train_orig[data_train_orig['contacts'].notnull()]['mean_salary'].describe()

,mean_salary
count,1.908500e+04
mean,9.562109e+04
std,1.106741e+05
min,2.000000e+01
25%,4.250000e+04
50%,6.750000e+04
75%,1.150000e+05
max,2.750000e+06


In [ ]:
data_train

,name,area,employer,snippet,schedule,description,experience,key_skills,specializations,region,mean_salary
0,Агент по недвижимости,"{'id': '2', 'name': 'Санкт-Петербург', 'url': ...","{'id': '541016', 'name': 'АРИН', 'url': 'https...",{'requirement': 'Уверенное знание компьютера‚ ...,"{'id': 'flexible', 'name': 'Гибкий график'}","<p>Мы ищем людей, которым не достаточно того, ...","{'id': 'noExperience', 'name': 'Нет опыта'}","[{'name': 'Жилая недвижимость'}, {'name': 'Акт...","[{'id': '20.20', 'name': 'Агент', 'profarea_id...",Санкт-Петербург,125000.0
1,IT-специалист/ техник,"{'id': '1130', 'name': 'Братск', 'url': 'https...","{'id': '1136625', 'name': 'СМП-38', 'url': 'ht...",{'requirement': 'Навыки. Коммуникативные навык...,"{'id': 'flyInFlyOut', 'name': 'Вахтовый метод'}",<ul> </ul> <p><em><strong>Навыки</strong></em>...,"{'id': 'between1And3', 'name': 'От 1 года до 3...",[],"[{'id': '1.172', 'name': 'Начальный уровень, М...",Иркутская область,50000.0
2,Старший/ведущий инженер-программист,"{'id': '76', 'name': 'Ростов-на-Дону', 'url': ...","{'id': '611346', 'name': 'САРМАТ, НПП', 'url':...",{'requirement': 'Высшее инженерно-техническое ...,"{'id': 'fullDay', 'name': 'Полный день'}",<strong>Обязанности:</strong> <ul> <li>Разрабо...,"{'id': 'between1And3', 'name': 'От 1 года до 3...","[{'name': 'Linux'}, {'name': 'Bash'}, {'name':...","[{'id': '1.221', 'name': 'Программирование, Ра...",Ростовская область,80000.0
3,Эксперт / методист приемной комиссии,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'id': '171255', 'name': 'РХТУ им. Д.И. Мендел...",{'requirement': 'Высшее образование. Опыт рабо...,"{'id': 'fullDay', 'name': 'Полный день'}",<p>Департамент информационных технологий РХТУ ...,"{'id': 'between1And3', 'name': 'От 1 года до 3...","[{'name': '1С программирование'}, {'name': 'Об...","[{'id': '3.150', 'name': 'Менеджмент продукта ...",Москва,120000.0
4,Табельщица/табельщик,"{'id': '1613', 'name': 'Энгельс', 'url': 'http...","{'id': '617150', 'name': 'Сигнал, группа компа...",{'requirement': 'Знание в совершенстве програм...,"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Обязанности:</strong></p> <ul> <li>...,"{'id': 'between1And3', 'name': 'От 1 года до 3...","[{'name': 'Составление отчетности'}, {'name': ...","[{'id': '18.142', 'name': 'Машиностроение', 'p...",Саратовская область,15000.0
...,...,...,...,...,...,...,...,...,...,...,...
30067,Руководитель отдела персонала,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'id': '127256', 'name': 'СИНЕРГИЯ', 'url': 'h...",{'requirement': 'Университетское образование в...,"{'id': 'fullDay', 'name': 'Полный день'}",<strong>Обязанности:</strong> <ul> <li>Управле...,"{'id': 'moreThan6', 'name': 'Более 6 лет'}","[{'name': 'Подбор персонала'}, {'name': 'Адапт...","[{'id': '9.738', 'name': 'Другое', 'profarea_i...",Москва,190000.0
30068,Специалист по развитию и обучению персонала,"{'id': '1641', 'name': 'Набережные Челны', 'ur...","{'id': '1068750', 'name': 'Brinex', 'url': 'ht...",{'requirement': 'Высшее образование. Опыт рабо...,"{'id': 'fullDay', 'name': 'Полный день'}",<p>Группа компаний &quot;Бринэкс&quot; приглаш...,"{'id': 'between1And3', 'name': 'От 1 года до 3...","[{'name': 'Управление временем'}, {'name': 'Ор...","[{'id': '6.254', 'name': 'Рекрутмент', 'profar...",Республика Татарстан,45000.0
30069,Инженер-программист,"{'id': '1399', 'name': 'Магнитогорск', 'url': ...","{'id': '824486', 'name': 'Магнитогорский метал...","{'requirement': 'Опыт работы с базами данных, ...","{'id': 'fullDay', 'name': 'Полный день'}",<strong>Обязанности:</strong> <ul> <li>Разрабо...,"{'id': 'between1And3', 'name': 'От 1 года до 3...","[{'name': 'SQL'}, {'name': 'Работа с базами да...","[{'id': '1.221', 'name': 'Программирование, Ра...",Челябинская область,75000.0
30070,Куратор проекта,"{'id': '3', 'name': 'Екатеринбург', 'url': 'ht...","{'id': '1947314', 'name': 'ANCOR', 'url': 'htt...",{'requirement': 'Опыт личных (очных) продаж от...,"{'id': 'flexible', 'name': 'Гибкий гра

In [ ]:
data_train_orig['schedule']

,schedule
0,"{'id': 'flexible', 'name': 'Гибкий график'}"
1,"{'id': 'flyInFlyOut', 'name': 'Вахтовый метод'}"
2,"{'id': 'fullDay', 'name': 'Полный день'}"
3,"{'id': 'fullDay', 'name': 'Полный день'}"
4,"{'id': 'fullDay', 'name': 'Полный день'}"
...,...
30067,"{'id': 'fullDay', 'name': 'Полный день'}"
30068,"{'id': 'fullDay', 'name': 'Полный день'}"
30069,"{'id': 'fullDay', 'name': 'Полный день'}"
30070,"{'id': 'flexible', 'name': 'Гибкий график'}"


In [ ]:
import json
import re

In [ ]:
def get_emp(x):
    res = re.findall(r"'name': [\"']([^']+)[\"']", x)
    if res == []:
        print(x)
    return res[0]

def get_name(x):
    return json.loads(x.replace("'", '"'))['name']

def data_process(data):
    data['type_schedule'] = data['schedule'].apply(get_name)

    data['city'] = data['area'].apply(get_name)

    data['name_employer'] = data['employer'].apply(get_emp)

    # data['has_contacts'] = data['contacts'].isna()
    # data['has_contacts'] = data['has_contacts'].replace({True: 1, False: 0})


    data['experience_tr'] = data['experience'].apply(get_name)

    return data

In [ ]:
data_train = data_process(data_train)

col_to_drop1 = ['schedule',
              'area',
              'employer',
              'experience']

data_train.drop(columns = col_to_drop1,
         inplace = True)

In [ ]:
data_train

,name,snippet,description,key_skills,specializations,region,mean_salary,type_schedule,city,name_employer,experience_tr
0,Агент по недвижимости,{'requirement': 'Уверенное знание компьютера‚ ...,"<p>Мы ищем людей, которым не достаточно того, ...","[{'name': 'Жилая недвижимость'}, {'name': 'Акт...","[{'id': '20.20', 'name': 'Агент', 'profarea_id...",Санкт-Петербург,125000.0,Гибкий график,Санкт-Петербург,АРИН,Нет опыта
1,IT-специалист/ техник,{'requirement': 'Навыки. Коммуникативные навык...,<ul> </ul> <p><em><strong>Навыки</strong></em>...,[],"[{'id': '1.172', 'name': 'Начальный уровень, М...",Иркутская область,50000.0,Вахтовый метод,Братск,СМП-38,От 1 года до 3 лет
2,Старший/ведущий инженер-программист,{'requirement': 'Высшее инженерно-техническое ...,<strong>Обязанности:</strong> <ul> <li>Разрабо...,"[{'name': 'Linux'}, {'name': 'Bash'}, {'name':...","[{'id': '1.221', 'name': 'Программирование, Ра...",Ростовская область,80000.0,Полный день,Ростов-на-Дону,"САРМАТ, НПП",От 1 года до 3 лет
3,Эксперт / методист приемной комиссии,{'requirement': 'Высшее образование. Опыт рабо...,<p>Департамент информационных технологий РХТУ ...,"[{'name': '1С программирование'}, {'name': 'Об...","[{'id': '3.150', 'name': 'Менеджмент продукта ...",Москва,120000.0,Полный день,Москва,РХТУ им. Д.И. Менделеева,От 1 года до 3 лет
4,Табельщица/табельщик,{'requirement': 'Знание в совершенстве програм...,<p><strong>Обязанности:</strong></p> <ul> <li>...,"[{'name': 'Составление отчетности'}, {'name': ...","[{'id': '18.142', 'name': 'Машиностроение', 'p...",Саратовская область,15000.0,Полный день,Энгельс,"Сигнал, группа компаний",От 1 года до 3 лет
...,...,...,...,...,...,...,...,...,...,...,...
30067,Руководитель отдела персонала,{'requirement': 'Университетское образование в...,<strong>Обязанности:</strong> <ul> <li>Управле...,"[{'name': 'Подбор персонала'}, {'name': 'Адапт...","[{'id': '9.738', 'name': 'Другое', 'profarea_i...",Москва,190000.0,Полный день,Москва,СИНЕРГИЯ,Более 6 лет
30068,Специалист по развитию и обучению персонала,{'requirement': 'Высшее образование. Опыт рабо...,<p>Группа компаний &quot;Бринэкс&quot; приглаш...,"[{'name': 'Управление временем'}, {'name': 'Ор...","[{'id': '6.254', 'name': 'Рекрутмент', 'profar...",Республика Татарстан,45000.0,Полный день,Набережные Челны,Brinex,От 1 года до 3 лет
30069,Инженер-программист,"{'requirement': 'Опыт работы с базами данных, ...",<strong>Обязанности:</strong> <ul> <li>Разрабо...,"[{'name': 'SQL'}, {'name': 'Работа с базами да...","[{'id': '1.221', 'name': 'Программирование, Ра...",Челябинская область,75000.0,Полный день,Магнитогорск,Магнитогорский металлургический комбинат,От 1 года до 3 лет
30070,Куратор проекта,{'requirement': 'Опыт личных (очных) продаж от...,<p><strong>На новый проект крупной российской ...,"[{'name': 'Пользователь ПК'}, {'name': 'Работа...","[{'id': '1.221', 'name': 'Программирование, Ра...",Свердловская область,30000.0,Гибкий график,Екатеринбург,ANCOR,От 1 года до 3 лет


In [ ]:
data_train.columns

Index(['name', 'snippet', 'description', 'key_skills', 'specializations',
       'region', 'mean_salary', 'type_schedule', 'city', 'name_employer',
       'experience_tr'],
      dtype='object')

In [ ]:
import ast
from gensim.models import Word2Vec

In [ ]:
# Функция для извлечения всех навыков из строки
def extract_skills(skill_str):
    skills_list = ast.literal_eval(skill_str)
    return [skill['name'] for skill in skills_list]

# Применяем функцию для каждого элемента в столбце "key_skills"
data_train['skills'] = data_train['key_skills'].apply(extract_skills)

# Обучаем модель Word2Vec на скиллах
model = Word2Vec(sentences=data_train['skills'], vector_size=100, window=5, min_count=1, workers=4)

# Функция для получения эмбеддинга для каждого скилла
def get_embeddings(skill_list):
    embs = [model.wv[skill] for skill in skill_list]
    return np.mean(embs, axis = 0)

# Применяем функцию, чтобы получить эмбеддинги для каждого набора скиллов
embeddings = data_train['skills'].apply(get_embeddings)

/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
data_train['key_skills']

,key_skills
0,"[{'name': 'Жилая недвижимость'}, {'name': 'Акт..."
1,[]
2,"[{'name': 'Linux'}, {'name': 'Bash'}, {'name':..."
3,"[{'name': '1С программирование'}, {'name': 'Об..."
4,"[{'name': 'Составление отчетности'}, {'name': ..."
...,...
30067,"[{'name': 'Подбор персонала'}, {'name': 'Адапт..."
30068,"[{'name': 'Управление временем'}, {'name': 'Ор..."
30069,"[{'name': 'SQL'}, {'name': 'Работа с базами да..."
30070,"[{'name': 'Пользователь ПК'}, {'name': 'Работа..."


In [ ]:
def extract_skills(skill_str):
    # Преобразуем строку в список словарей
    skills_list = ast.literal_eval(skill_str)
    # Извлекаем все значения 'name' из списка
    return [skill['name'] for skill in skills_list]

data_train['key_skills'].apply(extract_skills)

,key_skills
0,"[Жилая недвижимость, Активные продажи, Подбор ..."
1,[]
2,"[Linux, Bash, C/C++]"
3,"[1С программирование, Обновление конфигурации ..."
4,"[Составление отчетности, Работа с оргтехникой,..."
...,...
30067,"[Подбор персонала, Адаптация персонала, Управл..."
30068,"[Управление временем, Организация тренинга для..."
30069,"[SQL, Работа с базами данных, Базы данных]"
30070,"[Пользователь ПК, Работа в команде, Навыки про..."


In [ ]:
data_train['skills']

,skills
0,"[Жилая недвижимость, Активные продажи, Подбор ..."
1,[]
2,"[Linux, Bash, C/C++]"
3,"[1С программирование, Обновление конфигурации ..."
4,"[Составление отчетности, Работа с оргтехникой,..."
...,...
30067,"[Подбор персонала, Адаптация персонала, Управл..."
30068,"[Управление временем, Организация тренинга для..."
30069,"[SQL, Работа с базами данных, Базы данных]"
30070,"[Пользователь ПК, Работа в команде, Навыки про..."


In [ ]:
def extract_skills(skill_str):
    # Преобразуем строку в список словарей
    skills_list = ast.literal_eval(skill_str)
    # Извлекаем все значения 'name' из списка
    return [skill['name'] for skill in skills_list]

# Применяем функцию для каждого элемента в столбце "skills"
data_train['skills'] = data_train['key_skills'].apply(extract_skills)

# Извлекаем все уникальные навыки из всех записей
all_skills = set([skill for sublist in data_train['skills'] for skill in sublist])

# Создаем новые столбцы для каждого навыка

for skill in all_skills:
    data_train[skill] = data_train['skills'].apply(lambda x: 1 if skill in x else 0)

data_train

Streaming output truncated to the last 5000 lines.
<ipython-input-15-a92b5dfc7166>:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_train[skill] = data_train['skills'].apply(lambda x: 1 if skill in x else 0)
<ipython-input-15-a92b5dfc7166>:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_train[skill] = data_train['skills'].apply(lambda x: 1 if skill in x else 0)
<ipython-input-15-a92b5dfc7166>:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whi

,name,snippet,description,key_skills,specializations,region,mean_salary,type_schedule,city,name_employer,...,Публичные выступления,GitLab,3D Ceramics,Pentaho,Дизайн полиграфии,Программы лояльности для клиентов,Hr Policies,UIKit,CMS Bitrix,Managed k8s (GKE)
0,Агент по недвижимости,{'requirement': 'Уверенное знание компьютера‚ ...,"<p>Мы ищем людей, которым не достаточно того, ...","[{'name': 'Жилая недвижимость'}, {'name': 'Акт...","[{'id': '20.20', 'name': 'Агент', 'profarea_id...",Санкт-Петербург,125000.0,Гибкий график,Санкт-Петербург,АРИН,...,0,0,0,0,0,0,0,0,0,0
1,IT-специалист/ техник,{'requirement': 'Навыки. Коммуникативные навык...,<ul> </ul> <p><em><strong>Навыки</strong></em>...,[],"[{'id': '1.172', 'name': 'Начальный уровень, М...",Иркутская область,50000.0,Вахтовый метод,Братск,СМП-38,...,0,0,0,0,0,0,0,0,0,0
2,Старший/ведущий инженер-программист,{'requirement': 'Высшее инженерно-техническое ...,<strong>Обязанности:</strong> <ul> <li>Разрабо...,"[{'name': 'Linux'}, {'name': 'Bash'}, {'name':...","[{'id': '1.221', 'name': 'Программирование, Ра...",Ростовская область,80000.0,Полный день,Ростов-на-Дону,"САРМАТ, НПП",...,0,0,0,0,0,0,0,0,0,0
3,Эксперт / методист приемной комиссии,{'requirement': 'Высшее образование. Опыт рабо...,<p>Департамент информационных технологий РХТУ ...,"[{'name': '1С программирование'}, {'name': 'Об...","[{'id': '3.150', 'name': 'Менеджмент продукта ...",Москва,120000.0,Полный день,Москва,РХТУ им. Д.И. Менделеева,...,0,0,0,0,0,0,0,0,0,0
4,Табельщица/табельщик,{'requirement': 'Знание в совершенстве програм...,<p><strong>Обязанности:</strong></p> <ul> <li>...,"[{'name': 'Составление отчетности'}, {'name': ...","[{'id': '18.142', 'name': 'Машиностроение', 'p...",Саратовская область,15000.0,Полный день,Энгельс,"Сигнал, группа компаний",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30067,Руководитель отдела персонала,{'requirement': 'Университетское образование в...,<strong>Обязанности:</strong> <ul> <li>Управле...,"[{'name': 'Подбор персонала'}, {'name': 'Адапт...","[{'id': '9.738', 'name': 'Другое', 'profarea_i...",Москва,190000.0,Полный день,Москва,СИНЕРГИЯ,...,0,0,0,0,0,0,0,0,0,0
30068,Специалист по развитию и обучению персонала,{'requirement': 'Высшее образование. Опыт рабо...,<p>Группа компаний &quot;Бринэкс&quot; приглаш...,"[{'name': 'Управление временем'}, {'name': 'Ор...","[{'id': '6.254', 'name': 'Рекрутмент', 'profar...",Республика Татарстан,45000.0,Полный день,Набережные Челны,Brinex,...,0,0,0,0,0,0,0,0,0,0
30069,Инженер-программист,"{'requirement': 'Опыт работы с базами данных, ...",<strong>Обязанности:</strong> <ul> <li>Разрабо...,"[{'name': 'SQL'}, {'name': 'Работа с базами да...","[{'id': '1.221', 'name': 'Программирование, Ра...",Челябинская область,75000.0,Полный день,Магнитогорск,Магнитогорский металлургический комбинат,...,0,0,0,0,0,0,0,0,0,0
30070,Куратор проекта,{'requirement': 'Опыт личных (очных) продаж от...,<p><strong>На новый проект крупной российской ...,"[{'name': 'Пользователь ПК'}, {'name': 'Работа...","[{'id': '1.221', 'name': 'Программирование, Ра...",Свердловская область,30000.0,Гибкий график,Екатеринбург,ANCOR,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
['key_skills', 'skills',
               'snippet', 'specializations',
              'description']

['key_skills', 'skills', 'snippet', 'specializations', 'description']

In [ ]:
len(data_train['name'].unique())

13521

In [ ]:
len(data_train['name_employer'].unique())

12973

In [ ]:
data_train['experience_tr'].unique()

array(['Нет опыта', 'От 1 года до 3 лет', 'От 3 до 6 лет', 'Более 6 лет'],
      dtype=object)

In [ ]:
data_train['experience_tr'] = data_train['experience_tr'].replace({
    'Нет опыта': '0',
    'От 1 года до 3 лет': '1',
    'От 3 до 6 лет': '2',
    'Более 6 лет': 3
})

In [ ]:
data_train

,name,snippet,description,key_skills,specializations,region,mean_salary,type_schedule,city,name_employer,...,Бюджетные и автономные учреждения,Системы охранно-тревожной сигнализации,Enfocus PitStop,Социальный маркетинг,Оценка производительности,Проведение групповых уроков по плаванию,Вторичная недвижимость,Гражданское строительство,Nuxt.js,далион
0,Агент по недвижимости,{'requirement': 'Уверенное знание компьютера‚ ...,"<p>Мы ищем людей, которым не достаточно того, ...","[{'name': 'Жилая недвижимость'}, {'name': 'Акт...","[{'id': '20.20', 'name': 'Агент', 'profarea_id...",Санкт-Петербург,125000.0,Гибкий график,Санкт-Петербург,АРИН,...,0,0,0,0,0,0,1,0,0,0
1,IT-специалист/ техник,{'requirement': 'Навыки. Коммуникативные навык...,<ul> </ul> <p><em><strong>Навыки</strong></em>...,[],"[{'id': '1.172', 'name': 'Начальный уровень, М...",Иркутская область,50000.0,Вахтовый метод,Братск,СМП-38,...,0,0,0,0,0,0,0,0,0,0
2,Старший/ведущий инженер-программист,{'requirement': 'Высшее инженерно-техническое ...,<strong>Обязанности:</strong> <ul> <li>Разрабо...,"[{'name': 'Linux'}, {'name': 'Bash'}, {'name':...","[{'id': '1.221', 'name': 'Программирование, Ра...",Ростовская область,80000.0,Полный день,Ростов-на-Дону,"САРМАТ, НПП",...,0,0,0,0,0,0,0,0,0,0
3,Эксперт / методист приемной комиссии,{'requirement': 'Высшее образование. Опыт рабо...,<p>Департамент информационных технологий РХТУ ...,"[{'name': '1С программирование'}, {'name': 'Об...","[{'id': '3.150', 'name': 'Менеджмент продукта ...",Москва,120000.0,Полный день,Москва,РХТУ им. Д.И. Менделеева,...,0,0,0,0,0,0,0,0,0,0
4,Табельщица/табельщик,{'requirement': 'Знание в совершенстве програм...,<p><strong>Обязанности:</strong></p> <ul> <li>...,"[{'name': 'Составление отчетности'}, {'name': ...","[{'id': '18.142', 'name': 'Машиностроение', 'p...",Саратовская область,15000.0,Полный день,Энгельс,"Сигнал, группа компаний",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30067,Руководитель отдела персонала,{'requirement': 'Университетское образование в...,<strong>Обязанности:</strong> <ul> <li>Управле...,"[{'name': 'Подбор персонала'}, {'name': 'Адапт...","[{'id': '9.738', 'name': 'Другое', 'profarea_i...",Москва,190000.0,Полный день,Москва,СИНЕРГИЯ,...,0,0,0,0,0,0,0,0,0,0
30068,Специалист по развитию и обучению персонала,{'requirement': 'Высшее образование. Опыт рабо...,<p>Группа компаний &quot;Бринэкс&quot; приглаш...,"[{'name': 'Управление временем'}, {'name': 'Ор...","[{'id': '6.254', 'name': 'Рекрутмент', 'profar...",Республика Татарстан,45000.0,Полный день,Набережные Челны,Brinex,...,0,0,0,0,0,0,0,0,0,0
30069,Инженер-программист,"{'requirement': 'Опыт работы с базами данных, ...",<strong>Обязанности:</strong> <ul> <li>Разрабо...,"[{'name': 'SQL'}, {'name': 'Работа с базами да...","[{'id': '1.221', 'name': 'Программирование, Ра...",Челябинская область,75000.0,Полный день,Магнитогорск,Магнитогорский металлургический комбинат,...,0,0,0,0,0,0,0,0,0,0
30070,Куратор проекта,{'requirement': 'Опыт личных (очных) продаж от...,<p><strong>На новый проект крупной российской ...,"[{'name': 'Пользователь ПК'}, {'name': 'Работа...","[{'id': '1.221', 'name': 'Программирование, Ра...",Свердловская область,30000.0,Гибкий график,Екатеринбург,ANCOR,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
data_train.drop(['key_skills', 'skills'], axis=1, inplace=True)
data_train_f = data_train.drop(['snippet', 'description', 'specializations', 'region'], axis=1)
data_train_f['experience_tr'] =pd.to_numeric(data_train_f['experience_tr'])
data_train_f

,name,mean_salary,type_schedule,city,name_employer,experience_tr,Stock market,работа со статистикой,Сквозная аналитика,3. Навыки межличностного делового общения.,...,Публичные выступления,GitLab,3D Ceramics,Pentaho,Дизайн полиграфии,Программы лояльности для клиентов,Hr Policies,UIKit,CMS Bitrix,Managed k8s (GKE)
0,Агент по недвижимости,125000.0,Гибкий график,Санкт-Петербург,АРИН,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,IT-специалист/ техник,50000.0,Вахтовый метод,Братск,СМП-38,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Старший/ведущий инженер-программист,80000.0,Полный день,Ростов-на-Дону,"САРМАТ, НПП",1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Эксперт / методист приемной комиссии,120000.0,Полный день,Москва,РХТУ им. Д.И. Менделеева,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Табельщица/табельщик,15000.0,Полный день,Энгельс,"Сигнал, группа компаний",1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30067,Руководитель отдела персонала,190000.0,Полный день,Москва,СИНЕРГИЯ,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30068,Специалист по развитию и обучению персонала,45000.0,Полный день,Набережные Челны,Brinex,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30069,Инженер-программист,75000.0,Полный день,Магнитогорск,Магнитогорский металлургический комбинат,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30070,Куратор проекта,30000.0,Гибкий график,Екатеринбург,ANCOR,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X = data_train_f.drop(columns=["mean_salary"])
y = data_train_f["mean_salary"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

In [ ]:
X_train

,name,type_schedule,city,name_employer,experience_tr,Stock market,работа со статистикой,Сквозная аналитика,3. Навыки межличностного делового общения.,BOSCH,...,Публичные выступления,GitLab,3D Ceramics,Pentaho,Дизайн полиграфии,Программы лояльности для клиентов,Hr Policies,UIKit,CMS Bitrix,Managed k8s (GKE)
7274,Заместитель руководителя отдела мотивации и оп...,Полный день,Владивосток,Агентство КИТ Консалт,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4826,Web UI/UX-дизайнер (Middle),Полный день,Москва,Интеллектуальные Системы,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15965,Программист Битрикс,Полный день,Ростов-на-Дону,"Mibok, Internet Agency",1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13647,Агент по продажам недвижимости,Полный день,Сочи,Агентство недвижимости АТЛАС,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13177,Менеджер по персоналу (КДП),Полный день,Москва,Рк-Сервис,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28017,Риелтор,Гибкий график,Москва,ИНКОМ-НЕДВИЖИМОСТЬ,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17728,Агент по недвижимости (начинающий специалист),Полный день,Ковров,АЮВЕЛ НЕДВИЖИМОСТЬ,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29199,Координатор проекта в аутсорсинговую компанию,Полный день,Екатеринбург,Ресурс Менеджмент,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7293,Спасатель-инструктор,Полный день,Санкт-Петербург,Комплекс отдыха Семь Озер,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
categorical = list(X_train.dtypes[X_train.dtypes == "object"].index)
categorical

['name', 'type_schedule', 'city', 'name_employer']

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge


lin_reg = MyLinearRegression(weight_calc = 'sgd',
                           alpha=1e-2,
                           max_iter=5000,
                            early_stopping=0.0001 ,
                            lambda_1=0.01,
                            lambda_2=0.01,
                            regularization='l1l2',
                          batch_size = 32)

column_transformer = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first'), categorical)
    ],
    remainder='passthrough'  # Оставляем все остальные колонки (если они есть) без изменений
)

pipeline = Pipeline([
    ('transformer', column_transformer),
    ('regressor',  lin_reg)  # Используем ваш класс MyLinearRegression
])

In [ ]:
model = pipeline.fit(X_train, y_train)

100%|██████████| 5000/5000 [06:17<00:00, 13.25it/s]


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
y_train_pred = model.predict(X_train)
y_pred = model.predict(X_test)

print("Test MSE = %.4f" % mean_squared_error(y_test, y_pred))
print('Test MAE = %.4f' % mean_absolute_error(y_test, y_pred) )
print('Test R2 = %.4f' % r2_score(y_test, y_pred) )
print("Train MSE = %.4f" % mean_squared_error(y_train, y_train_pred))
print('Train MAE = %.4f' % mean_absolute_error(y_train, y_train_pred) )
print('Train R2 = %.4f' % r2_score(y_train, y_train_pred) )

X=  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
coeff=  [ 5.50538624e+01 -4.47009546e+01 -1.19145754e+02 ... -8.27352456e+01
 -2.90609384e+02  4.17969230e-05]
intercept=  45856.14964830249
type X=  <class 'numpy.ndarray'>
type coeff=  <class 'numpy.ndarray'>
type intercept=  <class 'numpy.float64'>
shape X=  (21050, 28727)
shape coeff=  (28727,)
shape intercept=  ()


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 2, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


X=  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
coeff=  [ 5.50538624e+01 -4.47009546e+01 -1.19145754e+02 ... -8.27352456e+01
 -2.90609384e+02  4.17969230e-05]
intercept=  45856.14964830249
type X=  <class 'numpy.ndarray'>
type coeff=  <class 'numpy.ndarray'>
type intercept=  <class 'numpy.float64'>
shape X=  (9022, 28727)
shape coeff=  (28727,)
shape intercept=  ()
Test MSE = 7033131578.2522
Test MAE = 41265.3624
Test R2 = 0.3431
Train MSE = 6907905178.5970
Train MAE = 40864.9362
Train R2 = 0.3744


In [ ]:
example = pd.read_csv('Sample_Submission.csv')
test_orig = pd.read_csv('for_prediction.csv')

In [ ]:
test_orig.drop(columns = ['Id'],
         inplace = True)

In [ ]:
test = test_orig.copy()

In [ ]:
class_drop = ['premium', 'department', 'has_test', 'response_letter_required',
                     'working_days', 'working_time_intervals', 'working_time_modes',
                     'immediate_redirect_url', 'published_at', 'created_at',
                     'response_url', 'sort_point_distance', 'archived', 'insider_interview',
                    'url', 'alternate_url', 'relations', 'type', 'address',
                    'accept_temporary']

test.drop(columns = class_drop,
         inplace = True)

In [ ]:
test =  data_process(test)

In [ ]:
def extract_skills(skill_str):
    # Преобразуем строку в список словарей
    skills_list = ast.literal_eval(skill_str)
    # Извлекаем все значения 'name' из списка
    return [skill['name'] for skill in skills_list]

# Применяем функцию для каждого элемента в столбце "skills"
test['skills'] = test['key_skills'].apply(extract_skills)

# Извлекаем все уникальные навыки из всех записей
all_skills = set([skill for sublist in test['skills'] for skill in sublist])

# Создаем новые столбцы для каждого навыка
for skill in all_skills:
    test[skill] = test['skills'].apply(lambda x: 1 if skill in x else 0)

Streaming output truncated to the last 5000 lines.
<ipython-input-81-f6aebeee25be>:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test[skill] = test['skills'].apply(lambda x: 1 if skill in x else 0)
<ipython-input-81-f6aebeee25be>:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test[skill] = test['skills'].apply(lambda x: 1 if skill in x else 0)
<ipython-input-81-f6aebeee25be>:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.

In [ ]:
col_to_drop1 = ['schedule',
              'area',
              'employer',
              'contacts',
              'experience']
col_to_drop2 = ['key_skills', 'skills',
               'snippet', 'specializations',
              'description']

test.drop(columns = col_to_drop1,
         inplace = True)

test.drop(columns = col_to_drop2,
         inplace = True)

In [ ]:
for col in X_train:
    if col not in test:
        test[col] = 0

Streaming output truncated to the last 5000 lines.
<ipython-input-83-1dd043347f69>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test[col] = 0
<ipython-input-83-1dd043347f69>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test[col] = 0
<ipython-input-83-1dd043347f69>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newfra

In [ ]:
test.dropna()

,name,region,type_schedule,city,name_employer,experience_tr,3D,Quarkus,NuxtJS,Финансовое планирование,...,Cycle,Единоборства,Yii 2,поиск кандидатов,Grid,Бюджетные и автономные учреждения,Enfocus PitStop,Оценка производительности,Проведение групповых уроков по плаванию,далион
0,Помощник маркетолога,Москва,Удаленная работа,Москва,EKO BIKE,От 1 года до 3 лет,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Менеджер по персоналу,Санкт-Петербург,Полный день,Санкт-Петербург,"Ваш дом, агентство недвижимости",Нет опыта,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Инструктор на детскую игровую площадку (ТЦ ""Ме...",Республика Башкортостан,Полный день,Уфа,Мегаполис,Нет опыта,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Программист Delphi,Новосибирская область,Удаленная работа,Новосибирск,AMS Software,От 3 до 6 лет,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ведущий специалист ВКС,Москва,Полный день,Москва,Технокад,От 1 года до 3 лет,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7513,HR менеджер,Республика Крым,Полный день,Симферополь,Группа компаний Shels (ООО Шелс),От 1 года до 3 лет,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7514,Руководитель HR отдела,Москва,Полный день,Москва,Edison,От 1 года до 3 лет,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7515,Системный аналитик (Стажер),Свердловская область,Гибкий график,Екатеринбург,Альфа-Банк,Нет опыта,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7516,Руководитель Отдела развития банковских технол...,Приморский край,Полный день,Владивосток,СКБ Приморья Примсоцбанк,От 1 года до 3 лет,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
test['experience_tr'] = test['experience_tr'].replace({
    'Нет опыта': '0',
    'От 1 года до 3 лет': '1',
    'От 3 до 6 лет': '2',
    'Более 6 лет': 3
})

In [ ]:
test[test.select_dtypes(include=np.number).columns] = test[test.select_dtypes(include=np.number).columns].astype('float')

In [ ]:
test.shape

(7518, 8703)

In [ ]:
y_pred = model.predict(X_test)

X=  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
coeff=  [ 6.40898981e+00  4.51827835e+01 -1.04806988e+02 ...  1.30814257e+02
 -5.41022106e+02 -5.35253235e-05]
intercept=  46499.21765033995
type X=  <class 'numpy.ndarray'>
type coeff=  <class 'numpy.ndarray'>
type intercept=  <class 'numpy.float64'>
shape X=  (9022, 28731)
shape coeff=  (28731,)
shape intercept=  ()


In [ ]:
predss = model.predict(test)

X=  [[0.0 0.0 0.0 ... 0.0 0.0 0.0]
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]
 ...
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]]
coeff=  [ 6.40898981e+00  4.51827835e+01 -1.04806988e+02 ...  1.30814257e+02
 -5.41022106e+02 -5.35253235e-05]
intercept=  46499.21765033995
type X=  <class 'numpy.ndarray'>
type coeff=  <class 'numpy.ndarray'>
type intercept=  <class 'numpy.float64'>
shape X=  (7518, 28731)
shape coeff=  (28731,)
shape intercept=  ()


In [ ]:
# ct = column_transformer

# ct.fit(X_train)

# test_ = ct.transform(test)
# test_.shape

(7518, 28731)

In [ ]:
example['Predicted'] = predss

In [ ]:
example.to_csv('predictions.csv', index = False)

In [ ]:
data_train['mean_salary'].mean()

97728.25091447194